In [3]:
!wget --header="Host: doc-00-14-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_glh8fmi9goppngvb244pvfpmluc342mq_nonce=h9uf3qegcvl26" --header="Connection: keep-alive" "https://doc-00-14-docs.googleusercontent.com/docs/securesc/1l3ul9726mftuj85bnpea3lnean3t71f/1pda3vh3rjaqjspasir6vp4i9n2g7aev/1639677225000/03167207420045571302/03167207420045571302/1QlWBLhERsMWePPtEsnzGm9YAtRqCOJbP?e=download&authuser=0&nonce=h9uf3qegcvl26&user=03167207420045571302&hash=9kbk8alut90nto5fvrfq8ll7j9im4vje" -c -O 'finalized_model.sav'

--2021-12-16 17:57:25--  https://doc-00-14-docs.googleusercontent.com/docs/securesc/1l3ul9726mftuj85bnpea3lnean3t71f/1pda3vh3rjaqjspasir6vp4i9n2g7aev/1639677225000/03167207420045571302/03167207420045571302/1QlWBLhERsMWePPtEsnzGm9YAtRqCOJbP?e=download&authuser=0&nonce=h9uf3qegcvl26&user=03167207420045571302&hash=9kbk8alut90nto5fvrfq8ll7j9im4vje
Resolving doc-00-14-docs.googleusercontent.com (doc-00-14-docs.googleusercontent.com)... 108.177.120.132, 2607:f8b0:4001:c18::84
Connecting to doc-00-14-docs.googleusercontent.com (doc-00-14-docs.googleusercontent.com)|108.177.120.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1177129152 (1.1G) [application/octet-stream]
Saving to: ‘finalized_model.sav’

finalized_model.sav 100%[===================>]   1.10G   146MB/s    in 11s     

2021-12-16 17:57:35 (107 MB/s) - ‘finalized_model.sav’ saved [1177129152/1177129152]



In [4]:
import pickle

In [32]:
#RandomForest, ExtraTrees, XGB
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import pandas_profiling as pan
import joblib
from sklearn.preprocessing import StandardScaler

In [6]:
#upload the csv files provided'  

In [48]:


#the input should be a  dataframe
def final(df):
    
    df = pd.read_csv(df, index_col=0)
    filename = 'finalized_model.sav'
    loaded_model = joblib.load(filename)
   # _____________________________
    df = df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Fuel_Price',"Temperature","CPI","Unemployment"],axis = 1)
    df['IsHoliday']= df['IsHoliday'].map({False:0,True:1})
    df['Type'] = df['Type'].map({'A':1,'B':2,'C':3})
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['week'] = df['Date'].dt.week
    dfy = df['Weekly_Sales']
    df =df.drop(['Date',"Weekly_Sales"],axis = 1)
    
    m =  loaded_model.predict(df)
    weights = df.IsHoliday.apply(lambda x: 5 if x else 1)
    j = np.round(np.sum(weights*abs(dfy-m))/(np.sum(weights)), 2)
    return m
    


In [49]:
final('/content/trial1.csv')

array([23547.85190816, 42433.41728869, 41326.02201786, ...,
         880.76163095,   890.04056548,   981.6058125 ])

In [44]:

#the input should be a csv file
def finaltwice(df):
    filename = 'finalized_model.sav'
    loaded_model = joblib.load(filename)
    df = pd.read_csv(df, index_col=0)
    df = df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Fuel_Price',"Temperature","CPI","Unemployment"],axis = 1)
    df['IsHoliday']= df['IsHoliday'].map({False:0,True:1})
    df['Type'] = df['Type'].map({'A':1,'B':2,'C':3})
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['week'] = df['Date'].dt.week
    dfy = df['Weekly_Sales']
    df =df.drop(['Date',"Weekly_Sales"],axis = 1)
    m =  loaded_model.predict(df)
    #the metric
    weights = df.IsHoliday.apply(lambda x: 5 if x else 1)
    j = np.round(np.sum(weights*abs(dfy-m))/(np.sum(weights)), 2)


    return j
    


In [47]:
WMAE_Score= finaltwice("/content/trial1.csv")
print(WMAE_Score)

651.2
